In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

class Capsule(layers.Layer):
    def __init__(self, num_capsules, capsule_dim, routings=3):
        super(Capsule, self).__init__()
        self.num_capsules = num_capsules
        self.capsule_dim = capsule_dim
        self.routings = routings
        self.W = self.add_weight(shape=[1, 48, 48, self.num_capsules * self.capsule_dim, 256], 
                                 initializer='glorot_uniform', name='W')

    def call(self, input):
        input = tf.expand_dims(input, -1)
        input = tf.tile(input, [1, 1, 1, self.num_capsules * self.capsule_dim])

        # Convolution operation for digit capsules
        input = tf.matmul(input, self.W)
        input = tf.reshape(input, [-1, 48, 48, self.num_capsules, self.capsule_dim])
        
        # Apply dynamic routing algorithm
        b = tf.zeros_like(input[:, :, :, :, 0])
        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=2)
            output = tf.reduce_sum(c[:, :, :, :, None] * input, axis=2)
            output = self.squash(output)
            if i < self.routings - 1:
                b += tf.reduce_sum(output[:, :, :, :, None] * input, axis=-1)

        return output

    def squash(self, input):
        norm = tf.norm(input, axis=-1, keepdims=True)
        norm_squared = norm ** 2
        scale = norm_squared / (1 + norm_squared)
        output = scale * input / norm
        return output

# Define the Capsule Network architecture
def CapsuleNetwork(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    conv1 = layers.Conv2D(64, (3, 3), activation='relu')(inputs)
    conv2 = layers.Conv2D(64, (3, 3), activation='relu')(conv1)
    conv3 = layers.Conv2D(128, (3, 3), activation='relu')(conv2)
    capsule = Capsule(num_capsules=num_classes, capsule_dim=16, routings=3)(conv3)
    flatten = layers.Flatten()(capsule)
    output = layers.Dense(num_classes, activation='softmax')(flatten)
    model = models.Model(inputs=inputs, outputs=output)
    return model

# Instantiate the Capsule Network
model = CapsuleNetwork(input_shape=(48, 48, 1), num_classes=7)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_val, y_val))

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)
